In [ ]:
#This Code is from Collab , it is the code that has given better result and added mood keywords into it

In [1]:
pip show tensorflow keras scikit-learn

Name: tensorflow
Version: 2.19.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: e:\ai ml models\.venv\lib\site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: 
---
Name: keras
Version: 3.9.2
Summary: Multi-backend Keras
Home-page: 
Author: 
Author-email: Keras team <keras-users@googlegroups.com>
License: Apache License 2.0
Location: e:\ai ml models\.venv\lib\site-packages
Requires: absl-py, h5py, ml-dtypes, namex, numpy, optree, packaging, rich
Required-by: keras-tuner, tensorflow
---
Name: scikit-learn
Version: 1.6.1
Summary: A set of python modules for machine learning and data mining
Home-page: https://sc

In [ ]:
import os
import nltk
import random
import json
import numpy as np
import pickle
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
from collections import Counter
from nltk.corpus import stopwords
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import load_model
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential , Model
from tensorflow.keras.layers import (
    Input, Dense, Dropout, LayerNormalization, Add, MultiHeadAttention,
    Bidirectional, LSTM, BatchNormalization, Layer
)
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam,AdamW
from tensorflow.keras.optimizers.schedules import ExponentialDecay

# Ensure NLTK stopwords are downloaded
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                                 initializer="normal", trainable=True)
        self.b = self.add_weight(name="att_bias", shape=(1,),
                                 initializer="zeros", trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b)
        a = K.softmax(e, axis=1)
        output = x * a
        return K.sum(output, axis=1)

# ---- Step 1: Load Dataset ----
with open("combined_dataset.json", "r") as f:
    lines = f.readlines()
parsed_data = [json.loads(line) for line in lines if line.strip()]

# Extract Context (X) and Response (Y)
X_texts = [sample["Context"] for sample in parsed_data]
Y_labels = [sample["Response"] for sample in parsed_data]

label_contexts = defaultdict(list)
for context, label in zip(X_texts, Y_labels):
    label_contexts[label].append(context)

# Extract top keywords for each mood label
mood_keywords = {}

for label, texts in label_contexts.items():
    # Filter out empty or whitespace-only texts
    clean_texts = [text.strip() for text in texts if text.strip()]

    # Skip if no valid texts
    if not clean_texts:
        print(f"⚠️ Skipping label '{label}' due to empty or invalid texts.")
        continue

    try:
        vectorizer = CountVectorizer(stop_words="english", max_features=15)
        X_vec = vectorizer.fit_transform(clean_texts)

        # Skip if no valid vocabulary extracted
        if not vectorizer.get_feature_names_out().size:
            print(f"⚠️ Skipping label '{label}' due to only stop words.")
            continue

        keywords = vectorizer.get_feature_names_out()
        mood_keywords[label] = list(keywords)

    except ValueError as e:
        print(f"⚠️ Skipping label '{label}' due to error: {e}")

# Save to JSON file
with open("mood_keywords.json", "w") as f:
    json.dump(mood_keywords, f, indent=4)

print("✅ Saved mood-based keywords to mood_keywords.json")

# ---- Step 2: Load Sentence-BERT Model for Better Embeddings ----
sbert_model = SentenceTransformer('paraphrase-MiniLM-L12-v2')

# Load or compute embeddings
try:
    X_emb = np.load("x_embeddings_sbert.npy")
    Y_encoded = np.load("y_encoded.npy")
    with open("response_encoder.pkl", "rb") as f:
        label_encoder = pickle.load(f)
    print("✅ Loaded saved embeddings and encoder.")
except FileNotFoundError:
    X_emb = sbert_model.encode(X_texts)  # Get Sentence-BERT embeddings

    label_encoder = LabelEncoder()
    Y_encoded = label_encoder.fit_transform(Y_labels)

    np.save("x_embeddings_sbert.npy", X_emb)
    np.save("y_encoded.npy", Y_encoded)
    with open("response_encoder.pkl", "wb") as f:
        pickle.dump(label_encoder, f)
    print("✅ Computed and saved embeddings and encoder.")

num_classes = len(label_encoder.classes_)
Y_encoded = to_categorical(Y_encoded, num_classes=num_classes)  # Convert to one-hot encoding

# Count instances per class
class_counts = Counter(np.argmax(Y_encoded, axis=1))
print(f"📊 Original Class Distribution: {class_counts}")

# **Remove classes with only 1 sample**
valid_classes = {cls for cls, count in class_counts.items() if count > 1}
valid_indices = [i for i, label in enumerate(np.argmax(Y_encoded, axis=1)) if label in valid_classes]

X_filtered = X_emb[valid_indices]
Y_filtered = Y_encoded[valid_indices]

# **Recalculate class distribution**
filtered_class_counts = Counter(np.argmax(Y_filtered, axis=1))
print(f"📊 Filtered Class Distribution: {filtered_class_counts}")

# Ensure no class has fewer than 2 samples
min_samples_per_class = min(filtered_class_counts.values())

if min_samples_per_class < 2:
    print("⚠️ Some classes still have only 1 sample after filtering. SMOTETomek will be skipped.")
    X_resampled, Y_resampled = X_filtered, Y_filtered  # Use filtered dataset
else:
    # Dynamically set k_neighbors based on min_samples_per_class
    smote_k = min(5, min_samples_per_class - 1)  # Ensure valid k_neighbors
    smote_k = max(smote_k, 1)  # Ensure at least 1 neighbor

    print(f"✅ Applying SMOTE with k_neighbors={smote_k} for class balancing...")

    smote = SMOTE(k_neighbors=smote_k, random_state=42)
    X_smote, Y_smote = smote.fit_resample(X_filtered, np.argmax(Y_filtered, axis=1))

    print("✅ Applying TomekLinks for noise reduction...")
    smote_tomek = SMOTETomek(random_state=42)
    X_resampled, Y_resampled = smote_tomek.fit_resample(X_smote, Y_smote)

    # Convert back to one-hot encoding
    Y_resampled = to_categorical(Y_resampled, num_classes=num_classes)
    print(f"📊 Class Distribution After SMOTETomek: {Counter(np.argmax(Y_resampled, axis=1))}")

# ✅ Splitting into train-test sets
num_samples = len(X_resampled)

# Ensure test size is valid
test_size = min(int(0.2 * num_samples), num_samples - num_classes)
test_size = max(test_size, num_classes)  # Ensure at least `num_classes` samples

X_train, X_test, Y_train, Y_test = train_test_split(
    X_resampled, Y_resampled, test_size=test_size, random_state=42, stratify=np.argmax(Y_resampled, axis=1)
)

print(f"✅ Training samples: {len(X_train)}, Testing samples: {len(X_test)}")

# ---- Step 5: Reshape Input for LSTM ----
X_train = np.expand_dims(X_train, axis=1)  # Add a time step dimension
X_test = np.expand_dims(X_test, axis=1)

Y_train = np.argmax(Y_train, axis=1)  # Convert labels to integers
Y_test = np.argmax(Y_test, axis=1)

# ---- Step 6: Compute Class Weights ----
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(Y_train),
    y=Y_train
)
class_weight_dict = dict(enumerate(class_weights))


In [ ]:
# ---- Step 7: Load or Define the Model ----
# ---- Check for Existing Model ----
model_path = "optimized_lstm_model.h5"

if os.path.exists(model_path):
    print("✅ Loading existing model for further training...")
    model = load_model(model_path, custom_objects={"AttentionLayer": AttentionLayer})

    # ✅ Reset the optimizer
    lr_schedule = ExponentialDecay(
        initial_learning_rate=0.0013,
        decay_steps=1150,
        decay_rate=0.97
    )
    optimizer = AdamW(learning_rate=3e-4)  # Create a new optimizer

    # ✅ Recompile the model with the new optimizer
    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=optimizer,
        metrics=["accuracy"]
    )
else:
    print("🚀 No previous model found. Training from scratch...")

    # ✅ Define learning rate scheduler here
    lr_schedule = ExponentialDecay(
        initial_learning_rate=0.001,
        decay_steps=1200,
        decay_rate=0.9
    )
# Transformer Block
    def transformer_block(inputs, num_heads=4, ff_dim=256, dropout_rate=0.4):
        attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=inputs.shape[-1])(inputs, inputs)
        attention_output = Dropout(dropout_rate)(attention_output)
        out1 = LayerNormalization(epsilon=1e-6)(Add()([inputs, attention_output]))

        ff_output = Dense(ff_dim, activation="relu")(out1)
        ff_output = Dense(inputs.shape[-1])(ff_output)
        ff_output = Dropout(dropout_rate)(ff_output)
        return LayerNormalization(epsilon=1e-6)(Add()([out1, ff_output]))

    # Input shape
    input_shape = (1, X_train.shape[-1])
    model_input = Input(shape=input_shape)

    # Transformer Block (expand to 3D)
    x = transformer_block(model_input)

    # BiLSTM Layers
    x = Bidirectional(LSTM(256, return_sequences=True))(x)
    x = Dropout(0.3)(x)
    x = BatchNormalization()(x)

    x = Bidirectional(LSTM(256, return_sequences=True))(x)
    x = Dropout(0.3)(x)
    x = BatchNormalization()(x)

    # Attention Layer
    x = AttentionLayer()(x)
    x = Dropout(0.3)(x)

    x = Dense(256, activation="relu")(x)
    x = Dropout(0.3)(x)

    output = Dense(num_classes, activation="softmax")(x)
    model = Model(inputs=model_input, outputs=output)

    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=Adam(learning_rate=lr_schedule,clipnorm=1.0),
        metrics=["accuracy"]
    )

# ---- Train the Model ----
early_stopping = EarlyStopping(monitor="val_loss", patience=9, restore_best_weights=True)

history = model.fit(
    X_train, Y_train,
    validation_data=(X_test, Y_test),
    epochs=60,
    batch_size=32,
    class_weight=class_weight_dict,
    callbacks=[early_stopping]
)

# ---- Save the current model temporarily for comparison ----
model.save("new_lstm_model.h5")
print("✅ New model saved temporarily as 'new_lstm_model.h5'")

# ---- Evaluate the Model ----
test_loss, test_acc = model.evaluate(X_test, Y_test)
print(f"✅ Optimized Model Test Accuracy: {test_acc * 100:.2f}%")

# ---- Step 9: Save the Best Model ----
if os.path.exists("optimized_lstm_model.h5"):
    prev_model = load_model("optimized_lstm_model.h5", custom_objects={"AttentionLayer": AttentionLayer})

    _, prev_model_acc = prev_model.evaluate(X_test, Y_test, verbose=0)
else:
    prev_model_acc = 0.0

# ---- Evaluate the new model ----
_, new_model_acc = model.evaluate(X_test, Y_test, verbose=0)

# ---- Compare and replace if improved ----
if new_model_acc > prev_model_acc:
    model.save("optimized_lstm_model.h5")
    print(f"✅ New model outperformed the previous one. Accuracy improved from {prev_model_acc*100:.2f}% → {new_model_acc*100:.2f}%")
else:
    print(f"⚠️ New model did NOT improve accuracy ({new_model_acc*100:.2f}%). Keeping previous model ({prev_model_acc*100:.2f}%).")


# ---- Step 10: Plot Training vs Validation Accuracy ----
plt.plot(history.history["accuracy"], label="Train Accuracy", color='blue')
plt.plot(history.history["val_accuracy"], label="Validation Accuracy", color='orange')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.title("Transformer + BiLSTM + Attention Training vs Validation Accuracy")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
import json
import numpy as np
from tensorflow.keras.models import load_model
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pickle
from tensorflow.keras.layers import Layer
import tensorflow.keras.backend as K

# ---- Custom Attention Layer (Used in Model) ----
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1),
                                 initializer="normal", trainable=True)
        self.b = self.add_weight(name="att_bias", shape=(1,),
                                 initializer="zeros", trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        e = K.tanh(K.dot(x, self.W) + self.b)
        a = K.softmax(e, axis=1)
        output = x * a
        return K.sum(output, axis=1)

# ---- Step 1: Load SBERT model for embeddings ----
sbert_model = SentenceTransformer('paraphrase-MiniLM-L12-v2')

# ---- Step 2: Load the trained mood classification model and label encoder ----
mood_model = load_model("optimized_lstm_model.h5", custom_objects={"AttentionLayer": AttentionLayer})

with open("response_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

# ---- Step 3: Load extracted questions JSON ----
with open("extracted_questions.json", "r") as f:
    questions_by_mood = json.load(f)

# ---- Step 4: Function to classify mood ----
def classify_mood(user_input):
    embedding = sbert_model.encode([user_input])
    input_tensor = np.expand_dims(embedding, axis=1)  # Shape: (1, 1, embedding_dim)
    prediction = mood_model.predict(input_tensor)
    predicted_class = np.argmax(prediction)
    try:
        return label_encoder.inverse_transform([predicted_class])[0]
    except:
        return "general"

# ---- Step 5: Extract all questions under the mood ----
def get_all_questions_for_mood(mood, json_data):
    mood = mood.capitalize()
    if mood not in json_data:
        return []
    
    mood_entries = json_data[mood]
    all_questions = []

    for entry in mood_entries:
        for section_name, section_data in entry.items():
            questions = section_data.get("questions", [])
            all_questions.extend(questions)
    
    return all_questions

# ---- Step 6: Find best-matching question ----
def fetch_best_matching_question(user_input):
    detected_mood = classify_mood(user_input)
    print(f"🧠 Detected Mood: {detected_mood}")

    all_questions = get_all_questions_for_mood(detected_mood, questions_by_mood)

    if not all_questions:
        return "❌ Sorry, I couldn't find questions for your mood."

    # Embed input and all questions
    input_embedding = sbert_model.encode([user_input])[0]
    question_embeddings = sbert_model.encode(all_questions)

    # Cosine similarity to find best match
    similarities = cosine_similarity([input_embedding], question_embeddings)[0]
    best_index = np.argmax(similarities)
    return all_questions[best_index]

# ---- Step 7: Run chatbot interaction loop ----
if __name__ == "__main__":
    print("💬 Chatbot ready! Type 'exit' to quit.\n")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("Chatbot: Take care. Goodbye! 👋")
            break
        response = fetch_best_matching_question(user_input)
        print("Chatbot:", response)
